# Корреляция 

Коэффициент корреляции выражает степень похожести двух наборов чисел. Важно помнить, что корреляция между наборами чисел --- не то же самое, что прямая зависимость между ними.

<center><b>Correlation is not Causation!</b></center>

![](https://tylervigen.com/correlation_project/correlation_images/number-people-who-drowned-by-falling-into-a-swimming-pool_number-of-films-niclas-cage-appeared-in.png)

*Пример взят [отсюда](https://tylervigen.com/spurious-correlations). Здесь же можно найти ссылки на интересные датасеты.*

Напомним определение коэффициента корреляции между наборами чисел $x = (x_1, \ldots, x_n)$ и $y = (y_1, \ldots, y_n)$:

$$
  \rho = \frac{(x_1 - \overline x)(y_1 - \overline y) + \ldots + (x_n - \overline x)(y_n - \overline y)}{se(x) \cdot se(y)} \\
  \overline x = \frac{x_1 + \ldots + x_n}{n} \\
  se(x) = \sqrt{(x_1 - \overline x)^2 + \ldots + (x_n - \overline x)^2} \\
  \overline y = \frac{y_1 + \ldots + y_n}{n} \\
  se(y) = \sqrt{(y_1 - \overline y)^2 + \ldots + (y_n - \overline y)^2}
$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# Функция для отображения набора точек.
# Объяснение, что здесь что будет через 2-3 семинара, потерпите :)
def plot_dependence(x, y, x_label='', y_label='', title='', grid_on=False):
    plt.plot(x, y, 'o')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    if not grid_on:
        plt.grid()

Для демонстрации эффекта зависимости в данных загрузим датасет с потреблением топлива различными видами транспорта:

In [ ]:
mpg_data = pd.read_csv(
    'auto-mpg.data',  # путь к таблице
    delim_whitespace=True,  # в данном случае разделитель между полями не запятая, а любой пробельный знак
    header=None,  # в файле с таблицей не записаны имена колонок, так что не нужно их считывать
    names = [
        'mpg', 'cylinders', 'displacement',
        'horsepower', 'weight', 'acceleration',
        'model_year', 'origin', 'name'
    ],  # вручную задаём имена колонок
    na_values='?',  # пропущенные значения в этом датасете заполнены знаками "?"
)
print(mpg_data.shape)
mpg_data.head()

Удалим пропуски:

In [ ]:
mpg_data = mpg_data.dropna(how='any')
print(mpg_data.shape)

Выведем корреляционную таблицу для всех признаков:

In [ ]:
mpg_data.corr()

Как её читать: на пересечении строки с именем `A` и столбца с именем `B` стоит коэффициент корреляции между признаками `A` и `B`.

Посмотрим на коэффициент корреляции между скоростью потребления топлива `mpg` (miles per gallon, галлон $\approx 3.785$ литра) с весом `weight`:

In [ ]:
print("Коэффициент корреляции: ", mpg_data.corr()['mpg']['weight'])

Наблюдаем сильную отрицательную корреляцию.

Что это значит:
- с увеличением веса уменьшается расстояние, которое можно проехать, потратив фиксированное количество топлива
- если вес транспортного средства большой, то одного галлона хватает на не очень большое количество миль
- если транспорт лёгкий, то на одном галлоне он сможет проехать большее расстояние, чем тяжёлый

Построим график зависимости для рассмотренных ранее столбцов `mpg` и `weight`:

In [ ]:
plot_dependence(mpg_data.weight, mpg_data.mpg, 'weight', 'MPG', 'Weight vs MPG dependence')

___
## Задание 

Рассмотрите другие пары признаков в датасете. Найдите среди них те, которые
- слабо коррелируют друг с другом (коэффициент корреляции по модулю $< 0.2$)
- имеют сильную положительную корреляцию

Выведите значения корреляции между найденными признаками:

In [ ]:
# YOUR CODE

# Линейная регрессия 

Попытаемся предсказать (объяснить) величину $y$ через набор числовых характеристик $x_1, \ldots, x_n$. Например, хотелось бы найти зависимость скорости потребления топлива автомобилем от других его характеристик.

Будем предполагать и надеяться, что величина $y$ зависит от этих характеристик линейно, т.е. каждый признак влияет на скорость с каким-то фиксированным весом, который мы (пока) не знаем:
$$y \approx \beta + \alpha_1x_1 + \alpha_2x_2+...+\alpha_nx_n$$

Чтобы в дальнейшем иметь возможность угадывать $y$ только по заданным $x_1, \ldots, x_n$, надо подобрать веса, близкие к реальным (реальные нам неизвестны).

Подбор весов происходит за счёт минимизации разности между реальными значениями и предсказанными.

___


Начнем с самого простого вида линейной регрессии, когда есть зависимость только от одного признака
$$y\approx\beta + \alpha x,$$
где $\alpha$ --- это коэффициент наклона прямой, $\beta$ --- коэффициент смещения.

Посмотрим ещё раз на зависимость потребления топлива от веса транспортного средства:

In [ ]:
print("Коэффициент корреляции: ", mpg_data.corr()['mpg']['weight'])

plot_dependence(mpg_data.weight, mpg_data.mpg, 'weight', 'MPG', 'Weight vs MPG dependence')

Чтобы найти наилучший вариант прямой, описывающей наши данные, будем использовать модель `LinearRegression` из библиотеки `sklearn`:

In [ ]:
from sklearn.linear_model import LinearRegression

# Выделяем в отдельную переменную целевую переменную и характеристики.
Y = mpg_data[['mpg']]
X = mpg_data[['weight']]

# Создание модели, которая будет подбирать веса для признаков.
model = LinearRegression()

# Просим модель подобрать веса для признаков.
model.fit(X, Y)

# Предсказываем значения с помощью модели.
Y_predicted = model.predict(X)

plot_dependence(X, Y)
plot_dependence(X, Y_predicted, 'weight', 'MPG', 'Weight vs MPG dependence', True)

Посмотрим, какие получились коэффициенты:

In [ ]:
print("Коэффициент уклона:    ", model.coef_[0])
print("Коэффициент смещения: ", model.intercept_)

Аналогичным образом можно обучить модель на всех признаках, а не только на одном. Но визуализировать это уже будет сложновато, так что понадобится какая-то метрика оценивания качества предсказаний.

В анализе данных есть два эпата: обучение модели и валидация.

- на этапе обучения модель видит и характеристики $x_1, \ldots, x_n$, и значения $y$, которые надо будет предсказывать в будущем, на этих данных настраиваются веса для характеристик

- на этапе валидации в модель подаются только характеристики (без значений $y$), которых не было на этапе обучения, и модель предсказывает значения $y$, опираясь на то, что "узнала" при обучении

Метрика `mean_squared_error` или `MSE` оценивает величину ошибки. Она вычисляется отдельно для предсказаний на обучении и на валидации.

---

Для начала подготовим данные:
- выделим в отдельную переменную то, что предсказывается, т.е. столбец `mpg`
- а ещё сохраним все остальные столбцы с числовыми значениями, которые будут нашими признаками

In [ ]:
Y = mpg_data[['mpg']]
X = mpg_data.drop(['mpg', 'name'], axis=1)
X.shape, Y.shape

Теперь подключим необходимые функции для разделения выборки на обучающую и тестовую, а также для оценивания качества модели:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# Разбиваем данные на обучающую часть, по которой модель будет подбирать коэффициенты,
# и валидационную (тестовую) часть, по которой можно оценить, насколько хорошо коэффициенты подобраны.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, random_state=128)

print('Размеры обучающей выборки: ', X_train.shape, Y_train.shape)
print('Размеры тестовой выборки: ', X_val.shape, Y_val.shape)

In [ ]:
# Создание модели, которая будет подбирать веса для признаков.
model = LinearRegression()
# Просим модель подобрать веса для признаков.
model.fit(X_train, Y_train)

# Предсказываем значения с помощью модели.
Y_train_predicted = model.predict(X_train)
Y_val_predicted = model.predict(X_val)

Можно посмотреть, что предсказала модель:

In [ ]:
Y_train_predicted[:10]

И сравнить с реальными значениями:

In [ ]:
Y_train[:10].values

In [ ]:
# Считаем метрику, показывающую, как сильно ошибается модель.
train_error = mean_squared_error(Y_train_predicted, Y_train)
val_error = mean_squared_error(Y_val_predicted, Y_val)

print('Ошибка на обучении:  \t', train_error)
print('Ошибка на валидации:\t', val_error)
print('Разница:\t\t', val_error - train_error)

---
---

Выведите все коэффициенты модели в следующем формате:
```
имя_признака коэффициент
```

In [ ]:
# YOUR CODE

Что можно сказать по этим коэффициентам о важности признаков?

*Спойлер: пока ничего сказать нельзя, можно подумать, почему :)*